In [9]:
import time

workflows = dict()
objects = []

rules_finished = False
with open('example_input') as f:
    for line in f:
        if len(line) == 1 and rules_finished is False:
            rules_finished = True
            continue
        if rules_finished is False:
            line = line.strip()
            name = line.split('{')[0]
            rules = line.split('{')[1][:-1]
            rules = rules.split(',')
            rules_to_save = []
            for rule in rules:
                new_rule = rule.split(":")
                if len(new_rule) > 1:
                    com_1 = new_rule[0][0]
                    com_2 = new_rule[0][1]
                    com_3 = int(new_rule[0][2:])
                    com_4 = new_rule[1]
                    rules_to_save.append([com_1, com_2, com_3, com_4])
                else:
                    rules_to_save.append([new_rule[0]])
            workflows[name] = rules_to_save   

        if rules_finished is True:
            line = line.strip()[1:-1].split(",")
            new_object = dict()
            for metric in line:
                var = metric[0]
                value = int(metric[2:])
                new_object[var] = value
            objects.append(new_object)

x_steps = [1, 4000]
m_steps = [1, 4000]
a_steps = [1, 4000]
s_steps = [1, 4000]

print("Przed opt:", len(workflows))
# for flow in workflows:
#     print(flow, workflows[flow])

one_more_time = True
flows_to_remove = set()
while one_more_time is True:
    one_more_time = False
    for flow in workflows:
        if workflows[flow][-1][0] != 'R' and workflows[flow][-1][0] != 'A':
            one_more_time = True
            flows_to_remove.add(workflows[flow][-1][0])
            remove_key = workflows[flow].pop()[0]
            for element in workflows[remove_key]:
                workflows[flow].append(element)
# print("To remove", flows_to_remove)

for element in flows_to_remove:
    remove_element = True
    for flow in workflows:
        for rule in workflows[flow]:
            if rule[-1] == element:
                remove_element = False
                break
        if remove_element is False:
            break
    if remove_element is True:
        del workflows[element]
    
while True:
    updated_list = False
    to_remove = set()
    for flow in workflows:
        destinations = set()
        for rule in workflows[flow]:
            destinations.add(rule[-1])
        if len(destinations) == 1:
            updated_list = True
            workflows[flow] = [rule[-1]]
            to_remove.add(flow)
    for flow in workflows:
        for rule in workflows[flow]:
            if rule[-1] in to_remove:
                rule[-1] = workflows[rule[-1]][0]
    for element in to_remove:
        del workflows[element]
    if updated_list is False:
        break

# for flow in workflows:
#     new_flow = []
#     for rule in workflows[flow]:
#         if len(rule) > 1 and rule[3] == workflows[flow][-1][0]:
#             continue
#         new_flow.append(rule)
#     workflows[flow] = new_flow

print("Po opt:", len(workflows))
for flow in workflows:
    print(flow, workflows[flow])

for flow in workflows:
    for rule in workflows[flow]:
        if rule[0] == 'x':
            if rule[1] == '<':
                x_steps.append(int(rule[2])-1)
                x_steps.append(int(rule[2]))
            if rule[1] == '>':
                x_steps.append(int(rule[2]))
                x_steps.append(int(rule[2])+1)
        if rule[0] == 'm':
            if rule[1] == '<':
                m_steps.append(int(rule[2])-1)
                m_steps.append(int(rule[2]))
            if rule[1] == '>':
                m_steps.append(int(rule[2]))
                m_steps.append(int(rule[2])+1)
        if rule[0] == 'a':
            if rule[1] == '<':
                a_steps.append(int(rule[2])-1)
                a_steps.append(int(rule[2]))
            if rule[1] == '>':
                a_steps.append(int(rule[2]))
                a_steps.append(int(rule[2])+1)
        if rule[0] == 's':
            if rule[1] == '<':
                s_steps.append(int(rule[2])-1)
                s_steps.append(int(rule[2]))
            if rule[1] == '>':
                s_steps.append(int(rule[2]))
                s_steps.append(int(rule[2])+1)

x_steps = sorted(x_steps)
m_steps = sorted(m_steps)
a_steps = sorted(a_steps)
s_steps = sorted(s_steps)

print("x_steps", x_steps)
print("m_steps", m_steps)
print("a_steps", a_steps)
print("s_steps", s_steps)

# workflow_numbers = dict()
# new_workflows = []
# new_workflows.append(workflows["in"])
# workflow_numbers["in"] = 0
# workflow_numbers['A'] = -1
# workflow_numbers['R'] = -2

# for key in workflows:
#     if key == "in":
#         continue
#     new_workflows.append(workflows[key])
#     workflow_numbers[key] = len(new_workflows) - 1

# for i in range(len(new_workflows)):
#     for rule in new_workflows[i]:
#         rule[-1] = workflow_numbers[rule[-1]]

total = 0

loop_no = 0
st = time.time()

for i in range(0, len(x_steps)-1, 2):
    print(loop_no / (len(x_steps)/2), time.time() - st)
    st = time.time()
    loop_no += 1
    x_mult = (x_steps[i+1]) - (x_steps[i]) + 1
    for j in range(0, len(m_steps)-1, 2):
        m_mult = (m_steps[j+1]) - (m_steps[j]) + 1
        for k in range(0, len(a_steps)-1, 2):
            a_mult = (a_steps[k+1]) - (a_steps[k]) + 1
            for l in range(0, len(s_steps)-1, 2):
                s_mult = (s_steps[l+1]) - (s_steps[l]) + 1
                # print("*********")
                # print("X", x_steps[i], x_steps[i+1], (x_steps[i+1]) - (x_steps[i]))
                # print("M", (m_steps[j]), (m_steps[j+1]), (m_steps[j+1]) - (m_steps[j]))
                # print("A", (a_steps[k]), (a_steps[k+1]), (a_steps[k+1]) - (a_steps[k]))
                # print("S", (s_steps[l]), (s_steps[l+1]), (s_steps[l+1]) - (s_steps[l]))
                object = dict()
                object['x'] = x_steps[i]
                object['m'] = m_steps[j]
                object['a'] = a_steps[k]
                object['s'] = s_steps[l]
                next_workflow = 'in'
                # next_workflow = 0
                while True:
                    if next_workflow == 'R':
                        break
                    if next_workflow == 'A':
                        total += x_mult * m_mult * a_mult * s_mult
                        break
                    for rule in workflows[next_workflow]:
                        next_workflow = None
                        if len(rule) == 1:
                            next_workflow = rule[0]
                            break
                        if rule[0] in object:
                            val = object[rule[0]]
                            threshold = rule[2]
                            if rule[1] == '<' and val < threshold:
                                next_workflow = rule[3]
                                break
                            if rule[1] == '>' and val > threshold:
                                next_workflow = rule[3]
                                break
                    
print(total)

Przed opt: 11
Po opt: 4
px [['a', '<', 2006, 'qkq'], ['m', '>', 2090, 'A'], ['s', '<', 537, 'R'], ['x', '>', 2440, 'R'], ['A']]
qkq [['x', '<', 1416, 'A'], ['x', '>', 2662, 'A'], ['R']]
in [['s', '<', 1351, 'px'], ['s', '>', 2770, 'A'], ['m', '<', 1801, 'hdj'], ['R']]
hdj [['m', '>', 838, 'A'], ['a', '>', 1716, 'R'], ['A']]
x_steps [1, 1415, 1416, 2440, 2441, 2662, 2663, 4000]
m_steps [1, 838, 839, 1800, 1801, 2090, 2091, 4000]
a_steps [1, 1716, 1717, 2005, 2006, 4000]
s_steps [1, 536, 537, 1350, 1351, 2770, 2771, 4000]
0.0 0.00036334991455078125
0.25 0.0001742839813232422
0.5 0.00016951560974121094
0.75 0.0001671314239501953
167409079868000
